In [1]:
%matplotlib inline
import pandas as pd
import os
import re
import shutil
import numpy as np
import warnings

Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 461 ('figure.figsize   : 5, 3   ## figure size in inches')
Duplicate key in file WindowsPath('C:/Users/maa/.matplotlib/stylelib/paper.mplstyle'), line 462 ('figure.dpi       : 100        ## figure dots per inch')


In [2]:
df = pd.read_csv('../students.csv')

In [3]:
df

,family name,name
0,Dahlén,Isak
1,Dahlqvist,Petrus
2,Danielsson,Isac
3,Durkin,Kirill
4,Dynevik,Philip
5,Edsbagge,Carl
6,Egeland,Karl
7,Ekdahl,Jonathan
8,El Masri,Mohammed Ali
9,Elm Jonsson,Benjamin


In [4]:
regexp=re.compile(r'^ *(.*)')
def clean_names(name):
    result=regexp.search(name)
    return result.group(1)

In [5]:
df['name']=df['name'].apply(func=clean_names)

In [6]:
def full_name(row):
    return '%s %s' % (row['name'],row['family name'])

In [7]:
df.index = df.apply(func=full_name, axis=1)
assert df.index.is_unique

In [8]:
def create_folder(row,root_path='students/assignments'):
    
    family_name = row['family name']
    name = row['name']
    dir_name = '%s_%s' % (family_name,name)
    path = os.path.abspath(os.path.join(root_path,dir_name))
    if not os.path.exists(path):
        os.mkdir(path)
    
    return path
    

## Create folders:

In [9]:
root_path='students/assignments'

if not os.path.exists(root_path):
    os.mkdir(root_path)

df['dir_path'] = df.apply(func=create_folder, axis=1, root_path=root_path)

## Rename Canvas data

In [10]:
# trying to match person with files like:
#bergströmolssonamanda_48073_750610_svenskPunktlighetsStatistik.txt

s = 'bergströmolssonamanda_48073_750610_svenskPunktlighetsStatistik.txt'

In [11]:
parts = s.split('_')
id_string=parts[0]
file_name = parts[-1]
id_string

'bergströmolssonamanda'

In [12]:
df_files = pd.DataFrame()
directory_name=r'students/submissions'
for file_name_raw in os.listdir(directory_name):
    s_=pd.Series(name=file_name_raw)
    
    parts = file_name_raw.split('_')
    s_['id_string']=parts[0]
    s_['file_name'] = parts[-1]
    s_['file_path_raw'] = os.path.join(directory_name,file_name_raw)
    df_files=df_files.append(s_)

c:\dev\tme136-programming-and-algorithms\venv\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


In [13]:
df_files.head()

,file_name,file_path_raw,id_string
abadaemanuel_20815_733299_deutschesPunktlichkeitsStatistik.txt,deutschesPunktlichkeitsStatistik.txt,students/submissions\abadaemanuel_20815_733299...,abadaemanuel
abadaemanuel_20815_733300_svenskPunktlighetsStatistik.txt,svenskPunktlighetsStatistik.txt,students/submissions\abadaemanuel_20815_733300...,abadaemanuel
abadaemanuel_20815_733301_punktlighet.m,punktlighet.m,students/submissions\abadaemanuel_20815_733301...,abadaemanuel
abadaemanuel_20815_733302_Rapport matlab Uppgift 2.pdf,Rapport matlab Uppgift 2.pdf,students/submissions\abadaemanuel_20815_733302...,abadaemanuel
abdadeler_34710_754940_matlab1.pdf,matlab1.pdf,students/submissions\abdadeler_34710_754940_ma...,abdadeler


In [14]:
def find_id(row):

    family_names=row['family name'].split(' ')
    names=row['name'].split(' ')
    
    df_search=df_files.copy()
    
    searches_name = [name.lower() for name in names]
    searches_family = [family_name.lower() for family_name in family_names]
    searches = searches_name + searches_family
    
    for search in searches:
    
        mask = df_files['id_string'].str.contains(search)
        df_search=df_search.loc[mask]
    
    assert len(df_search['id_string'].unique())==1
    id_string=df_search.iloc[0]['id_string']
    return id_string

In [15]:
df['id_string'] = df.apply(func=find_id, axis=1)

## Copy and rename files

In [16]:
def copy_rename(row):
    id_string=row['id_string']
    mask=df_files['id_string']==id_string
    df_ = df_files.loc[mask]
    for index, sub_row in df_.iterrows():
        destination = os.path.join(row['dir_path'],sub_row['file_name'])
        if not os.path.exists(destination):
            shutil.copyfile(sub_row['file_path_raw'], destination)
    

In [17]:
df.apply(func=copy_rename, axis=1);

In [18]:
def make_clickable(val):
    s=r'file:///' + val
    return '<a href="{}">{}</a>'.format(s,"path")

In [19]:
def show(df):
    return df.style.format({'dir_path': make_clickable})

## Check files exist:

In [26]:
correct_code_src_dir=r'matlab'
src_files = [file for file in os.listdir(correct_code_src_dir) if os.path.splitext(file)[-1] == '.m']
assert len(src_files) > 0
    

def has_files(row, src_files):    
    
    for src_file in src_files:
        path=os.path.join(row['dir_path'],src_file)
        if not os.path.exists(path):
            return False
        
    return True

In [30]:
df['has_files'] = df.apply(func=has_files, src_files=src_files, axis=1)

In [32]:
mask=~df['has_files']
show(df.loc[mask])

,family name,name,dir_path,id_string,has_files
Karl Egeland,Egeland,Karl,path,egelandkarl,False
Jonathan Ekdahl,Ekdahl,Jonathan,path,ekdahljonathan,False
Benjamin Elm Jonsson,Elm Jonsson,Benjamin,path,elmjonssonbenjamin,False
Jonathan Erneborg,Erneborg,Jonathan,path,erneborgjonathan,False
David Espedalen,Espedalen,David,path,espedalendavid,False
Fabian Fagerlind,Fagerlind,Fabian,path,fagerlindfabian,False
Bekir Fazlija,Fazlija,Bekir,path,fazlijabekir,False
Sofia Friberg,Friberg,Sofia,path,fribergsofia,False
Emelie Gillerstedt,Gillerstedt,Emelie,path,gillerstedtemelie,False


In [42]:
def repair_file_names(row, src_files):
    
    for src_file in src_files:
        src_file_name, src_file_ext = os.path.splitext(src_file)
        for file in os.listdir(row['dir_path']):
            file_name, file_ext = os.path.splitext(file)
            if ((src_file_ext==file_ext) and
                (src_file_name != file_name) and
                (src_file_name in file_name)):
                src = os.path.join(row['dir_path'], file)
                dst = os.path.join(row['dir_path'], src_file)
                shutil.copyfile(src,dst)

In [46]:
df.apply(func=repair_file_names, axis=1, src_files=src_files)

df['has_files'] = df.apply(func=has_files, src_files=src_files, axis=1)
mask=~df['has_files']
show(df.loc[mask])

,family name,name,dir_path,id_string,has_files


In [48]:
show(df)

,family name,name,dir_path,id_string,has_files
Isak Dahlén,Dahlén,Isak,path,dahlénisak,True
Petrus Dahlqvist,Dahlqvist,Petrus,path,dahlqvistpetrus,True
Isac Danielsson,Danielsson,Isac,path,danielssonisac,True
Kirill Durkin,Durkin,Kirill,path,durkinkirill,True
Philip Dynevik,Dynevik,Philip,path,dynevikphilip,True
Carl Edsbagge,Edsbagge,Carl,path,edsbaggecarl,True
Karl Egeland,Egeland,Karl,path,egelandkarl,True
Jonathan Ekdahl,Ekdahl,Jonathan,path,ekdahljonathan,True
Mohammed Ali El Masri,El Masri,Mohammed Ali,path,elmasrimohammedali,True
Benjamin Elm Jonsson,Elm Jonsson,Benjamin,path,elmjonssonbenjamin,True


## Running the scripts to get the times

In [47]:
import matlab.engine
eng = matlab.engine.start_matlab()

#r = eng.punktlighet()

eng.quit()

In [17]:
def runner(row, eng):
    
    print('evaluate: %s %s' % (row['name'],row['family name']))
    run_script_name = 'student_AutomaticGearbox.m'
    
    if os.path.exists(run_script_name):
        os.remove(run_script_name)  # Remove old.
    
    assert not os.path.exists(run_script_name)
    
    for file in os.listdir(row['dir_path']):
        ext = os.path.splitext(file)[-1]
        if ext=='.m' or ext=='.mat':
            src=os.path.join(row['dir_path'],file)
            dst=file
            shutil.copyfile(src,dst)
    
    assert os.path.exists(run_script_name)
    
    r = eng.runner()
    finishing_time=r.pop('finishing_time')
    result = r
    #finishing_time=result['TimeTotal'][0][-1]
    return finishing_time
    

This one got stuck in an infinite loop beacuse Throttle is not set. **"Warning: You aren't pressing the throttle. This will stop the car. A car that is stationary won't win dragraces. You
are probably watching the simulation stuck in an infinite loop right now "**

In [18]:
def manual(full_name:str, ok:bool, comment:str, finishing_time=None):
    df.loc[full_name,'ok'] = ok
    df.loc[full_name,'manual'] = True
    df.loc[full_name,'comment'] = comment
    df.loc[full_name,'finishing_time'] = finishing_time

In [19]:
manual(full_name='Emelie Gillerstedt', ok=False, 
       comment='This one got stuck in an infinite loop beacuse Throttle is not set')

manual(full_name='Rasmus Fischer', ok=False, 
       comment='Incomplete if clause if, elseif but no else')



In [20]:
import matlab.engine
eng = matlab.engine.start_matlab()

for index, row in df.iterrows():
    
    if row['manual'] is True:
        print('(Handled manually)')
        continue  # This one is handled manually
    
    if row['ok'] is True:
        print('(ok, already checked)')
        continue
    
    try:
        finishing_time = runner(row=row, eng=eng)
    except Exception as e:
        fail = True
        df.loc[index,'finishing_time'] = np.NaN
        print('Fail!')
    else:
        fail = False
        df.loc[index,'finishing_time'] = finishing_time
        df.loc[index,'ok'] = True
            
eng.quit()

evaluate: Isak Dahlén
evaluate: Petrus Dahlqvist
evaluate: Isac Danielsson
evaluate: Kirill Durkin
evaluate: Philip Dynevik
evaluate: Carl Edsbagge
evaluate: Karl Egeland
evaluate: Jonathan Ekdahl
evaluate: Mohammed Ali El Masri
evaluate: Benjamin Elm Jonsson
evaluate: Johan Elnertz
evaluate: Martin Engelsen
evaluate: Love Engh
evaluate: Daniel Eriksson
evaluate: Vigg Eriksson
evaluate: Johan Erlandsson
evaluate: Jonathan Erneborg
evaluate: David Espedalen
evaluate: Fabian Fagerlind
evaluate: Hilda Fahl
evaluate: Bekir Fazlija
(Handled manually)
evaluate: Sofia Friberg
(Handled manually)
evaluate: Milton Göransson
evaluate: Anna Götberg
evaluate: Frida Gotenius
evaluate: Elin Gryte
evaluate: Christoffer Gustafsson Blüme
evaluate: Alexander Gustafsson
evaluate: May Venke Gylseth


In [21]:
eng.quit()

In [22]:
show(df)

,family name,name,dir_path,has_run_file,ok,manual,comment,finishing_time
Isak Dahlén,Dahlén,Isak,path,True,True,nan,nan,15.295810
Petrus Dahlqvist,Dahlqvist,Petrus,path,True,True,nan,nan,15.401637
Isac Danielsson,Danielsson,Isac,path,True,True,nan,nan,15.401637
Kirill Durkin,Durkin,Kirill,path,True,True,nan,nan,15.042472
Philip Dynevik,Dynevik,Philip,path,True,True,nan,nan,15.042472
Carl Edsbagge,Edsbagge,Carl,path,True,True,nan,nan,15.470123
Karl Egeland,Egeland,Karl,path,True,True,nan,nan,15.470123
Jonathan Ekdahl,Ekdahl,Jonathan,path,True,True,nan,nan,15.268968
Mohammed Ali El Masri,El Masri,Mohammed Ali,path,True,True,nan,nan,15.268968
Benjamin Elm Jonsson,Elm Jonsson,Benjamin,path,True,True,nan,nan,15.381043


In [23]:
df.sort_values(by='finishing_time', inplace=True)
show(df)

,family name,name,dir_path,has_run_file,ok,manual,comment,finishing_time
Kirill Durkin,Durkin,Kirill,path,True,True,nan,nan,15.042472
Philip Dynevik,Dynevik,Philip,path,True,True,nan,nan,15.042472
Anna Götberg,Götberg,Anna,path,True,True,nan,nan,15.052847
Frida Gotenius,Gotenius,Frida,path,True,True,nan,nan,15.052847
Hilda Fahl,Fahl,Hilda,path,True,True,nan,nan,15.119196
Fabian Fagerlind,Fagerlind,Fabian,path,True,True,nan,nan,15.119196
David Espedalen,Espedalen,David,path,True,True,nan,nan,15.261515
Jonathan Erneborg,Erneborg,Jonathan,path,True,True,nan,nan,15.261515
Elin Gryte,Gryte,Elin,path,True,True,nan,nan,15.261515
Christoffer Gustafsson Blüme,Gustafsson Blüme,Christoffer,path,True,True,nan,nan,15.261515


In [24]:
df.to_csv('Martin_Alexandersson_finishing_times.csv', sep=';', encoding='cp1252')